In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf, to_date
from pyspark.sql.types import FloatType
import matplotlib.pyplot as plt
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext

In [2]:
print(pyspark.__version__)
 


3.2.4


In [3]:
df = (spark.read
      .format("mongo")
      .option("spark.mongodb.input.uri", "mongodb://localhost:27017")
      .option("spark.mongodb.input.database", "TrumpDB")
      .option("spark.mongodb.input.collection", "TrumpCollection")
      .option("spark.mongodb.input.partitioner", "MongoSinglePartitioner")
      .load())


In [4]:
print(df.columns)

['_id', 'candidate_id', 'created_at', 'device', 'favorite_count', 'id', 'inserted_at', 'lang', 'polarity', 'retweet_count', 'state', 'subjectivity', 'tw_user_id', 'tweet_id', 'tweet_text', 'updated_at']


In [5]:
df = df.select('candidate_id', "created_at", "tweet_text")

In [ ]:
print(df.count())

In [ ]:
#df.show()

In [6]:
df_no_nulls = df.na.drop()
tweets = df_no_nulls.filter(df.candidate_id == 2)

In [ ]:
#tweets.show()

In [7]:
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    return analyzer.polarity_scores(text)["compound"]

sentiment_udf = udf(analyze_sentiment, FloatType())
spark.udf.register("sentiment_udf", sentiment_udf)

<function __main__.analyze_sentiment(text)>

In [8]:
# Convert timestamp to date
tweets = tweets.withColumn("date", to_date(col("created_at")))

# Calculate sentiment score for each tweet
tweets = tweets.withColumn("sentiment", sentiment_udf(col("tweet_text")))


In [9]:
tweets = tweets.select('tweet_text','date','sentiment')

In [ ]:
#tweets.show()

In [10]:
from pyspark.sql.functions import year

# filter out out-of-range dates
tweets = tweets.filter((year("date") >= 1) & (year("date") <= 9999))

In [11]:
daily_sentiment_df = tweets.groupBy("date") \
    .agg(F.mean("sentiment").alias("avg_sentiment")) \
    .orderBy("date")

In [ ]:
daily_sentiment_df.count()

In [ ]:
daily_sentiment_pd = daily_sentiment_df.toPandas()

In [ ]:
daily_sentiment_pd

In [ ]:
plt.plot(daily_sentiment_pd['date'],daily_sentiment_pd['avg_sentiment'])